In [38]:
import numpy as np
import pandas as pd

import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint
# just set the seed for the random number generator
np.random.seed(107)

import matplotlib.pyplot as plt
import yfinance as yf

In [39]:
data_path = 'd:\\Google Drive Backup\\FIN 512\\Data'

In [40]:
sic_tick = pd.read_excel(data_path+'\\SIC and Ticker.xlsx').dropna()
sic_tick = sic_tick[sic_tick['HSICCD'].astype(str).str.contains('283')]
ticks_283 = ' '.join(sic_tick['HTICK'].values)

In [123]:
data = yf.download(ticks_283, start="2000-01-01", end="2009-12-31")

[*********************100%***********************]  200 of 200 completed

49 Failed downloads:
- MDCO: No data found, symbol may be delisted
- MLND: Data doesn't exist for startDate = 946706400, endDate = 1262239200
- STML: No data found, symbol may be delisted
- FENC: Data doesn't exist for startDate = 946706400, endDate = 1262239200
- LJPC: Data doesn't exist for startDate = 946706400, endDate = 1262239200
- BPMX: No data found, symbol may be delisted
- AXGT: Data doesn't exist for startDate = 946706400, endDate = 1262239200
- XTNT: Data doesn't exist for startDate = 946706400, endDate = 1262239200
- RDUS: Data doesn't exist for startDate = 946706400, endDate = 1262239200
- CBAY: Data doesn't exist for startDate = 946706400, endDate = 1262239200
- TGTX: Data doesn't exist for startDate = 946706400, endDate = 1262239200
- ZSAN: Data doesn't exist for startDate = 946706400, endDate = 1262239200
- ELAN: Data doesn't exist for startDate = 946706400, endDate = 1262239200
- ZTS: Data doesn

In [129]:
adj_close_data = data[('Adj Close')].dropna(axis=1,how='all')#.isna().sum()>10
adj_close_data = adj_close_data[[i for i,j in pd.Series(adj_close_data.isna().sum(axis=0)==2).items() if j==True]]

In [138]:
def find_cointegrated_pairs(data):
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    scores = []
    pvalues = []
    count = 0
    for i in range(n):
        for j in range(i+1, n):
            S1 = data[keys[i]]
            S2 = data[keys[j]]
            result = coint(S1, S2)
            scores.append((keys[i], keys[j],result[0]))
            pvalues.append((keys[i], keys[j],result[1]))
            count+=1
            if count%1000==0:
                print(str(count)+' Pairs done..!!')
    return scores,pvalues

In [139]:
coint_values = find_cointegrated_pairs(adj_close_data.dropna())

1000 Pairs done..!!
2000 Pairs done..!!
3000 Pairs done..!!
4000 Pairs done..!!


In [148]:
pd.pivot(pd.DataFrame(coint_values[1]),index=0,columns=1).to_csv('pvalues.csv')

In [111]:
len(pairs)

1535

In [21]:
import pandas as pd

pvalues = pd.read_csv('final pvalues.csv',sep=',')

In [137]:
top_mark = ['JNJ', 'PFE', 'MRK', 'TEVA', 'AMGN', 'BMY', 'LLY', 'GILD', 'BIIB',
       'BHC', 'MYL', 'ALXN', 'VRTX', 'PRGO', 'QGEN', 'IDXX', 'UTHR', 'BMRN',
       'REGN', 'TECH']
pvalues[pvalues['company A'].isin(top_mark)].sort_values('pvalue').to_csv('filtered_pairs.csv')

In [92]:
def get_marketcap(tick):
    return pd.read_csv('https://fmpcloud.io/api/v3/historical-market-capitalization/'+tick+'?datatype=csv&period=quarter&apikey=4236aeb9edb3d547b97701ace0027d75')

In [93]:
market_caps = {tick:get_marketcap(tick) for tick in pvalues['company A'].unique()}

In [105]:
market_caps['NYMX'][market_caps['NYMX']['date']=='2010-12-30']

,date,marketCap
2481,2010-12-30,409716540.0


In [82]:
for tick in market_caps.keys():
    market_caps[tick]['date'] = pd.to_datetime(market_caps[tick]['date']).dt.to_period('Y')

In [111]:
mark_cap_2010 = {tick:market_caps[tick][market_caps[tick]['date']=='2010-12-30']['marketCap'].values for tick in pvalues['company A'].unique() if len(market_caps[tick][market_caps[tick]['date']=='2010-12-30']['marketCap'].values)!=0}

In [122]:
pd.DataFrame(mark_cap_2010).transpose()

,0
BBI,1.463700e+08
ABIO,2.426760e+07
ARMP,2.292189e+14
GERN,4.979532e+08
SRPT,2.426112e+08
...,...
TTNP,7.279200e+07
BSTC,1.828028e+08
WST,1.555704e+09
VIVO,9.744440e+08


In [138]:
pd.DataFrame(mark_cap_2010).transpose().sort_values(0,ascending=False).drop(['ARMP','BCRX']).to_csv('market_cap.csv')